### Subetapa 2.1.1: Lidar com dados nulos. Por exemplo, preencher valores nulos de notas com 0 ou a média.
### Subetapa 2.1.2: Normalizar as colunas de texto (nomes de cursos, etc.) para evitar inconsistências (por exemplo, converter para letras maiúsculas ou minúsculas).
### Subetapa 2.1.3: Validar e corrigir os tipos de dados. Garantir que a coluna de notas seja do tipo numérico (float ou int).

In [4]:
import pandas as pd

dataset = '../data/student_grades.csv'
df = pd.read_csv(dataset)

In [6]:
df.fillna(0) # Preenche todos os valores nulos com "0" 
# Apesar de já sabermos pela análise anterior que não temos valores nulos.

,Student ID,Course,Grade
0,1,Math,F
1,1,Science,C
2,1,History,A
3,1,English,F
4,1,Physical Education,C
...,...,...,...
49995,10000,Math,D
49996,10000,Science,F
49997,10000,History,B
49998,10000,English,F


In [8]:
df['Course'].unique() # Verifica quais são os cursos que temos

array(['Math', 'Science', 'History', 'English', 'Physical Education'],
      dtype=object)

In [10]:
dc_course = { # Cria um dicionario para traduzir as matérias
    'Math': 'matematica',
    'Science': 'ciencias',
    'History': 'historia',
    'English': 'ingles',
    'Physical Education': 'educacao fisica'
}


df['Course'].replace(dc_course) # Testa a implementação, mas não aplica
# df.head() retorna para o ingles, portanto precisamos salvar as modificações
df['Course'] = df['Course'].replace(dc_course) # Agora sim, salvamos as mudanças
# df.head() retorna os valores atualizados


In [12]:
# Mesmo procedimento com as notas agora
df['Grade'].unique() 

array(['F', 'C', 'A', 'B', 'D'], dtype=object)

In [14]:
dc_grade = { # Cria dicionário para notas, trocando para valores numéricos
    'A':10,
    'B':8,
    'C':6,
    'D':4,
    'E':2,
    'F':0
}

df['Grade'] = df['Grade'].replace(dc_grade) # Atualizando novos valores
df.head() 

,Student ID,Course,Grade
0,1,matematica,0
1,1,ciencias,6
2,1,historia,10
3,1,ingles,0
4,1,educacao fisica,6


In [16]:
# Agora vamos mudar o texto das colunas em si.
dc_col_names = {
    'Student ID': 'IdAluno',
    'Course': 'curso',
    'Grade': 'nota'
}

df = df.rename(columns= dc_col_names) # Renomeia as colunas

df.head()

,IdAluno,curso,nota
0,1,matematica,0
1,1,ciencias,6
2,1,historia,10
3,1,ingles,0
4,1,educacao fisica,6


In [18]:
df.info() # Percebi que a coluna de notas ainda estava no formato de object, precisamos corrigir isso.
# As notas devem ser número de ponto flutuante.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   IdAluno  50000 non-null  int64 
 1   curso    50000 non-null  object
 2   nota     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [20]:
df['nota'] = df['nota'].astype(float) # Convertendo o tipo da coluna para float
df.info() # Verifica a mudança de object para float
df.head() # verifica a mudança 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   IdAluno  50000 non-null  int64  
 1   curso    50000 non-null  object 
 2   nota     50000 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.1+ MB


,IdAluno,curso,nota
0,1,matematica,0.0
1,1,ciencias,6.0
2,1,historia,10.0
3,1,ingles,0.0
4,1,educacao fisica,6.0


### Subetapa 2.2.1: Criar uma conexão com o banco de dados SQLite dentro do seu Notebook.
### Subetapa 2.2.2: Definir o nome da tabela que irá receber os dados, por exemplo, tabela_notas.
### Subetapa 2.2.3: Utilizar o pandas para carregar o DataFrame já tratado para a tabela SQLite.
#### df_tratado.to_sql('tabela_notas', conn, if_exists='replace', index=False)
### Subetapa 2.2.4: Verificar se a tabela foi criada e populada corretamente, executando uma simples consulta SQL (SELECT * FROM tabela_notas LIMIT 5).

In [22]:
import sqlite3 as sql

In [24]:
conn = sql.connect('../db/dev.db') 
# Cria a conexão com o banco de dados sqlite na pasta db, gerando autmomaticamente o dev.db

In [26]:
df.to_sql('tabela_notas', conn, if_exists='replace', index=False) # Converte nosso DataFrame do arquivo csv em uma tabela no banco de dados SQLite

# 'tabela_notas': nome para a tabela.
# conn: variável que guarda a conexão com o banco de dados.
# if_exists='replace': diz ao pandas o que fazer se a tabela já existir, ele vai substituir a tabela.
# index=False: O pandas, por padrão, tenta salvar o índice do DataFrame como uma coluna no banco de dados. 
# o índice não tem nenhum significado especial, no nosso exemplo, então vou ignorá-lo.

50000

In [28]:
cursor = conn.cursor() # cursor é a ferramenta que permite executar comandos SQL

cursor.execute("SELECT * FROM tabela_notas LIMIT 5").fetchall() # Testando se a população foi feita corretamente

[(1, 'matematica', 0.0),
 (1, 'ciencias', 6.0),
 (1, 'historia', 10.0),
 (1, 'ingles', 0.0),
 (1, 'educacao fisica', 6.0)]

In [30]:
cursor.execute("SELECT * FROM tabela_notas LIMIT 5") # necessário rodar de novo sem o fetchall para retornar o print corretamente
# Porque?
# O fetchall funciona assim:
# Imagine que o cursor é uma caixa de correio.
# Quando você usa o .execute(), é como se você tivesse enviado uma carta (SELECT * FROM tabela_notas) para os correios (SQLite). A carta pede um pacote com as 5 primeiras notas.
# Os correios enviam o pacote para a sua caixa de correio (cursor). O pacote está lá, mas ainda está fechado.
# Quando você usa o .fetchall(), é como se você estivesse abrindo o pacote e tirando todas as correspondências de uma só vez.
# Portanto o fetchall esvazia a sua caixa de correios (cursor).
resultados = cursor.fetchall() 
print(resultados)

[(1, 'matematica', 0.0), (1, 'ciencias', 6.0), (1, 'historia', 10.0), (1, 'ingles', 0.0), (1, 'educacao fisica', 6.0)]


# Fase 3 A partir daqui

### Subetapa 3.1.1: Criar uma nova célula no Notebook para cada consulta SQL.
### Subetapa 3.1.2: Escrever e executar as consultas para responder às perguntas-chave do projeto.
    * Qual a média de notas por curso?
    * Quais alunos tiveram a maior média?
    * Qual a distribuição de notas em uma matéria específica?
### Subetapa 3.1.3: Anotar os resultados e as conclusões de cada consulta.

### Subetapa 3.2.1: Adicionar comentários detalhados ao longo do Notebook explicando o que cada bloco de código faz.
### Subetapa 3.2.2: Escrever uma introdução e uma conclusão para o projeto, explicando o objetivo, a metodologia e os resultados obtidos.
### Subetapa 3.2.3: Salvar o Notebook final e o banco de dados SQLite para uso futuro.

In [31]:
cursor.execute("SELECT curso, AVG(nota) as media_notas FROM tabela_notas GROUP BY curso") # Media de notas por curso 

resultados = cursor.fetchall() 
print(resultados)

[('ciencias', 5.5618), ('educacao fisica', 5.5518), ('historia', 5.5656), ('ingles', 5.5968), ('matematica', 5.6316)]


In [46]:
cursor.execute("SELECT IdAluno, AVG(nota) as media_notas_por_aluno FROM tabela_notas GROUP BY IdAluno") # Vemos todas as notas por alunos

resultados = cursor.fetchall() 
print(resultados)

[(1, 4.4), (2, 6.8), (3, 7.2), (4, 4.8), (5, 6.8), (6, 5.6), (7, 5.2), (8, 4.8), (9, 5.6), (10, 3.6), (11, 6.4), (12, 6.0), (13, 8.0), (14, 7.2), (15, 6.4), (16, 7.6), (17, 4.4), (18, 4.4), (19, 6.4), (20, 3.6), (21, 8.0), (22, 4.0), (23, 4.8), (24, 6.8), (25, 4.8), (26, 9.6), (27, 6.0), (28, 6.4), (29, 8.0), (30, 5.6), (31, 4.4), (32, 4.8), (33, 4.8), (34, 5.6), (35, 2.8), (36, 4.8), (37, 7.6), (38, 5.2), (39, 7.6), (40, 5.6), (41, 6.8), (42, 4.8), (43, 6.0), (44, 5.6), (45, 2.0), (46, 4.8), (47, 5.2), (48, 7.6), (49, 6.4), (50, 3.6), (51, 6.4), (52, 5.2), (53, 4.4), (54, 6.0), (55, 8.0), (56, 5.2), (57, 6.4), (58, 3.6), (59, 6.8), (60, 4.4), (61, 5.6), (62, 4.0), (63, 6.4), (64, 3.2), (65, 4.8), (66, 5.2), (67, 6.0), (68, 6.4), (69, 5.6), (70, 8.0), (71, 4.0), (72, 5.6), (73, 6.0), (74, 4.4), (75, 5.6), (76, 6.8), (77, 5.6), (78, 6.4), (79, 6.8), (80, 6.8), (81, 6.4), (82, 3.6), (83, 5.6), (84, 5.6), (85, 6.4), (86, 6.8), (87, 4.4), (88, 6.4), (89, 1.6), (90, 6.0), (91, 6.8), (92, 7.

In [47]:

cursor.execute("SELECT IdAluno, AVG(nota) as media_notas_por_aluno FROM tabela_notas GROUP BY IdAluno ORDER BY media_notas_por_aluno DESC") # Agora ordenados pela média de notas

resultados = cursor.fetchall() 
print(resultados)

[(8749, 10.0), (6933, 10.0), (8886, 9.6), (8370, 9.6), (8287, 9.6), (8238, 9.6), (8121, 9.6), (7064, 9.6), (6927, 9.6), (6737, 9.6), (6597, 9.6), (5881, 9.6), (5386, 9.6), (4265, 9.6), (3864, 9.6), (2950, 9.6), (2553, 9.6), (1937, 9.6), (1809, 9.6), (673, 9.6), (190, 9.6), (26, 9.6), (9895, 9.2), (9863, 9.2), (9617, 9.2), (9555, 9.2), (9313, 9.2), (9292, 9.2), (9047, 9.2), (8896, 9.2), (8313, 9.2), (8054, 9.2), (7994, 9.2), (7682, 9.2), (7375, 9.2), (7204, 9.2), (7203, 9.2), (7110, 9.2), (6937, 9.2), (6891, 9.2), (6634, 9.2), (6385, 9.2), (6316, 9.2), (6083, 9.2), (6072, 9.2), (6054, 9.2), (5837, 9.2), (5803, 9.2), (5801, 9.2), (5594, 9.2), (5494, 9.2), (4978, 9.2), (3748, 9.2), (3357, 9.2), (3142, 9.2), (2602, 9.2), (2595, 9.2), (2292, 9.2), (1607, 9.2), (1402, 9.2), (1137, 9.2), (689, 9.2), (451, 9.2), (398, 9.2), (9693, 8.8), (9538, 8.8), (9518, 8.8), (9503, 8.8), (9464, 8.8), (9427, 8.8), (9250, 8.8), (9104, 8.8), (9086, 8.8), (9059, 8.8), (8949, 8.8), (8758, 8.8), (8704, 8.8), (86

In [51]:
cursor.execute("SELECT nota, count(nota) FROM tabela_notas WHERE curso = 'matematica' GROUP BY nota") 

resultados = cursor.fetchall() 
print(resultados)

[(0.0, 1924), (4.0, 2095), (6.0, 1953), (8.0, 2031), (10.0, 1997)]
